In [1]:
import json, nltk
import regex as re
import pandas as pd
import spacy
from spacy import displacy
from pathlib import Path
nlp = spacy.load('en_core_web_sm')
import numpy as np
import csv
from collections import defaultdict

In [ ]:
#Saving cleaned learner/corrector pairs into different output files
with open('082019_lang8_sp_eng_learner_cor_sents.txt') as file:
    with open('20191010_clean_l8_learner_sents.txt', 'w') as output:
        with open('20191010_clean_l8_cor_sents.txt', 'w') as output_2:
            for line in file:
                line = line.strip('\n')
                line = line.split('\t')
                #if len(line) == 1:
                    #print(line)
                output.write(str(line[0])+ '\n')
                output_2.write(str(line[1]+ '\n'))

In [4]:
with open("20191010_clean_l8_learner_sents.txt") as fd:
    file = fd.read().splitlines()
    
with open("20191010_clean_l8_cor_sents.txt") as fd_2:
    file_2 = fd_2.read().splitlines()
    
with open("20191012_l8_sp_eng_m2") as fd_3:
    file_3 = fd_3.read().splitlines()
    
#Aligning Learner sentences with feedback (No correction sentence == Duplicating Learner Sentence)
uncor_num = 0
cor_num = 0
l_c_dict = {l:c for l,c in zip(file, file_2)}
for k in l_c_dict:
    #Captures text enclosed in []
    a = re.search(r'\[(.*)\]', str(l_c_dict[k]))
    if a:
        l_c_dict[k] = a.group(1)
    if bool(l_c_dict[k]) == False:
        l_c_dict[k] = k
    else:
        pass
#Checking for number of Non-Corrections vs. Corrections
for k,v in l_c_dict.items():
    if k==v:
        uncor_num+=1
    else:
        cor_num+=1
print('Number of corrections: ', cor_num, cor_num/len(l_c_dict.keys())*100)
print('Number uncorrected: ', uncor_num, uncor_num/len(l_c_dict.keys()) *100)

Number of corrections:  10819 41.65158806544755
Number uncorrected:  15156 58.34841193455246


In [7]:
#Using spaCy in order to make a list of items containing dependencies and Part of speech (POS)
#information for each token in every learner/corrector pair
dep_file = []
dep_file_2 = []

for line in file:
    dep_sentence = nlp(line)
    deps = []
    for token in dep_sentence:
        l_tok = token.text + '_' + token.pos_ + '_' + token.dep_
        deps.append(l_tok)
    dep_file.append(deps)
    deps = []

for line_2 in file_2:
    dep_sentence_2 = nlp(line_2)
    deps_2 = []
    for token_2 in dep_sentence_2:
        c_tok = token_2.text + '_' + token_2.pos_ + '_' + token_2.dep_
        deps_2.append(c_tok)
    dep_file_2.append(deps_2)
    deps_2 = []

In [8]:
#Handles cases in which extracted learner/corrector sentence was empty
for item in range(0, len(file) -1):
    if file[item] == ' ':
        file.pop(item)
        file_2.pop(item)
        dep_file.pop(item)
        dep_file_2.pop(item)

In [10]:
e_list = []
currlist = []
#counter = 0

for line in open('20191012_l8_sp_eng_m2'):
    line = line.rstrip()
    if line == '':
        if currlist != []:
            #print(currlist)
            e_list.append(currlist)
            #counter += 1
        currlist = []
        continue
    currlist.append(line)

In [11]:
e_list[:10]

[["S he's still waiting for her to come back",
  'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ["S He wasn't able to remember their first kiss.",
  'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ['S She would have done.', 'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ["S He couldn't remember the first time they made love.",
  'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ['S She would have done.', 'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ["S He wasn't able to even remember her voice, her smile, her smell...",
  'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ['S She would have done.', 'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ['S The only thing he was able to remember after loosing her was her hands.',
  'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ['S The', 'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0'],
 ['S way she used to touch softly his face.',
  'A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0']]

In [12]:
m_l_object = [list(pair) for pair in zip(file, dep_file, file_2,dep_file_2,e_list)]

In [22]:
m_l_object[55]

['The silence is surrounding her and she can help trembling while thinks about him.',
 ['The_DET_det',
  'silence_NOUN_nsubj',
  'is_AUX_aux',
  'surrounding_VERB_ROOT',
  'her_PRON_dobj',
  'and_CCONJ_cc',
  'she_PRON_nsubj',
  'can_VERB_aux',
  'help_VERB_ROOT',
  'trembling_VERB_xcomp',
  'while_SCONJ_mark',
  'thinks_VERB_advcl',
  'about_ADP_prep',
  'him_PRON_pobj',
  '._PUNCT_punct'],
 "The silence is surrounding her, and she can't help trembling while she thinks about him.",
 ['The_DET_det',
  'silence_NOUN_nsubj',
  'is_AUX_aux',
  'surrounding_VERB_ROOT',
  'her_PRON_dobj',
  ',_PUNCT_punct',
  'and_CCONJ_cc',
  'she_PRON_nsubj',
  'ca_VERB_aux',
  "n't_PART_neg",
  'help_VERB_conj',
  'trembling_VERB_xcomp',
  'while_SCONJ_mark',
  'she_PRON_nsubj',
  'thinks_VERB_advcl',
  'about_ADP_prep',
  'him_PRON_pobj',
  '._PUNCT_punct'],
 ['S The silence is surrounding her and she can help trembling while thinks about him.',
  'A 4 5|||R:OTHER|||her,|||REQUIRED|||-NONE-|||0',
  "A 7

In [15]:
data = m_l_object

with open('20200129_l8_eng_sp_error_output.json', 'w') as outfile:
    json.dump(data, outfile)

In [18]:
with open('20200129_l8_eng_sp_error_output.json') as json_file:
    m_l_object = json.load(json_file)

In [23]:
#Target pronoun list that will be stored and tracked for in a dictionary
pronouns = '(I|me|we|us|he|him|she|her|it|they|them)'
#preps = '(on|in|for|with)'
pro_types = '(M:PRON|R:PRON|U:PRON)'
#prep_types = '(M:PREP|R:PREP|U:PREP)'
pronoun_type_counter = defaultdict(lambda: defaultdict(int))
pronoun_counter = defaultdict(int)


for item in m_l_object:
    #learner sentence
    l_t = str(item[0])
    #continue if not right part of corpus (== will yield corrected, >0 uncorrected)
    #if annot_sents[l_t] == 0:
            #continue
    #learner sentence dependencies
    l_d = item[1]
    #corrector sentence
    c_t = str(item[2])
    #corrector sentence dependencies
    c_d = item[3]
    
    for e in item[4]:
        pro_target = re.search(r'\|\|\|' + pro_types + r'\|\|\|' 
                               + r'.*\b(?i)' + pronouns + r'\b.*\|\|\|', e)
        #print(l_t)
        if pro_target:
            anno_type = pro_target.group(1)
            #print(anno_type)
            current_pronoun = pro_target.group(2).lower()
            #print(l_t, current_pronoun, e)
            #pronoun_type_counter[anno_type] += 1
            pronoun_type_counter[anno_type][current_pronoun] += 1
            #pronoun_counter[current_pronoun] += 1
            #if anno_type == 'U:PRON' and current_pronoun == 'it':
                #print(e)
          
#for p in pronoun_counter.keys():
    #print(p, pronoun_counter[p])
for a in pronoun_type_counter.keys():
    total = 0
    for p in pronoun_type_counter[a].keys():
        total += pronoun_type_counter[a][p]
        print(a,p,pronoun_type_counter[a][p])
    print(a, 'total: ', total)

M:PRON she 15
M:PRON i 88
M:PRON it 293
M:PRON they 27
M:PRON he 16
M:PRON we 11
M:PRON me 18
M:PRON him 2
M:PRON them 18
M:PRON us 3
M:PRON total:  491
R:PRON it 109
R:PRON i 115
R:PRON we 10
R:PRON me 13
R:PRON they 46
R:PRON he 12
R:PRON her 12
R:PRON them 29
R:PRON she 10
R:PRON us 2
R:PRON total:  358
U:PRON it 1
U:PRON i 1
U:PRON total:  2


In [ ]:
pronoun_type_counter['M:PRON'].keys()

In [24]:
pn_l8 = pd.DataFrame(m_l_object)
pn_l8.head()

,0,1,2,3,4
0,he's still waiting for her to come back,"[he_PRON_nsubj, 's_AUX_aux, still_ADV_advmod, ...",he's still waiting for her to come back,"[he_PRON_nsubj, 's_AUX_aux, still_ADV_advmod, ...","[S he's still waiting for her to come back, A ..."
1,He wasn't able to remember their first kiss.,"[He_PRON_nsubj, was_AUX_ROOT, n't_PART_neg, ab...",He wasn't able to remember their first kiss.,"[He_PRON_nsubj, was_AUX_ROOT, n't_PART_neg, ab...",[S He wasn't able to remember their first kiss...
2,She would have done.,"[She_PRON_nsubj, would_VERB_aux, have_AUX_aux,...",She would have done.,"[She_PRON_nsubj, would_VERB_aux, have_AUX_aux,...","[S She would have done., A -1 -1|||noop|||-NON..."
3,He couldn't remember the first time they made ...,"[He_PRON_nsubj, could_VERB_aux, n't_PART_neg, ...",He couldn't remember the first time they made ...,"[He_PRON_nsubj, could_VERB_aux, n't_PART_neg, ...",[S He couldn't remember the first time they ma...
4,She would have done.,"[She_PRON_nsubj, would_VERB_aux, have_AUX_aux,...",She would have done.,"[She_PRON_nsubj, would_VERB_aux, have_AUX_aux,...","[S She would have done., A -1 -1|||noop|||-NON..."


In [ ]:
#how many learner sentences were corrected
#total number of learner sentences
l_sents = defaultdict(int)
#annotated sentences
annot_sents = defaultdict(int)
#total_annotations
total_annots = 0

for i in range(0, len(m_l_object)):
    s = m_l_object[i][0]
    l_sents[s]+=1
    if i == 6623 or i == 9415 or i == 24820:
        continue
    no_op_search = re.search(r'\|\|\|(noop)\|\|\|', m_l_object[i][4][1])
    if not no_op_search:
        annot_sents[s]+=1
        total_annots += 1
    
#Unique Learner sentences
print(len(l_sents.keys()))
#Learner sentences with annotations
print(len(annot_sents.keys()))
#Total annotations
print(total_annots)

print(10850/25974 *100)

print(12883/10850)

In [ ]:
pronoun_counter = defaultdict(int)

#target pronoun list
pronouns = '(I|me|we|us|he|him|she|her|it|they|them)'

pro_target = re.search(r'\|\|\|(M:PRON)\|\|\|.*' + '\b(?i)' + pronouns + '\b.*\|\|\|', l_t)


#Learner pronoun usage over all sentences
pro_counter_l = 0
pro_counter_l_i = 0
pro_counter_l_me = 0
pro_counter_l_you = 0
pro_counter_l_it = 0
pro_counter_l_it_nsubj = 0
pro_counter_l_it_nsubjpass = 0
pro_counter_l_it_dobj = 0
pro_counter_l_he = 0
pro_counter_l_she = 0
pro_counter_l_him = 0
pro_counter_l_her = 0
pro_counter_l_we = 0
pro_counter_l_they = 0
pro_counter_l_them = 0
pro_counter_l_us = 0

#Pronoun counts in Corrector Sentences
pro_counter_c_m = 0

pro_counter_c = 0
pro_counter_c_i = 0
pro_counter_c_me = 0
pro_counter_c_you = 0
pro_counter_c_it = 0
pro_counter_c_it_nsubj = 0
pro_counter_c_it_nsubjpass = 0
pro_counter_c_it_dobj = 0
pro_counter_c_he = 0
pro_counter_c_she = 0
pro_counter_c_him = 0
pro_counter_c_her = 0
pro_counter_c_we = 0
pro_counter_c_they = 0
pro_counter_c_them = 0
pro_counter_c_us = 0

#stats on insertions
#all pronoun insertions
m_pron_counter = 0

i_insert = 0
i_insert_nsubj = 0

me_insert = 0
me_insert_nsubj = 0
me_insert_dobj = 0

you_insert = 0
you_insert_nsubj = 0
you_insert_dobj = 0

it_insert = 0
it_insert_nsubj = 0
it_insert_dobj = 0

me_insert = 0

he_insert = 0
he_insert_nsubj = 0
he_insert_dobj = 0

she_insert = 0
she_insert_nsubj = 0
she_insert_dobj = 0

him_insert = 0
him_insert_nsubj = 0
him_insert_dobj = 0

her_insert = 0
her_insert_nsubj = 0
her_insert_dobj = 0

we_insert = 0
we_insert_nsubj = 0
we_insert_dobj = 0

us_insert = 0
us_insert_nsubj = 0
us_insert_dobj = 0

they_insert = 0
they_insert_nsubj = 0
they_insert_dobj = 0

them_insert = 0
them_insert_nsubj = 0
them_insert_dobj = 0

for item in m_l_object:
    #learner sentence
    l_t = str(item[0])
    #continue if not right part of corpus (== will yield corrected, >0 uncorrected)
    if annot_sents[l_t] == 0:
            continue
    #learner sentence dependencies
    l_d = item[1]
    #corrector sentence
    c_t = str(item[2])
    #corrector sentence dependencies
    c_d = item[3]
    
    for e in item[4]:
            it_search = re.search(r'^A (\d+) (\d+)\|\|\|([^\|]+)\|\|\|([^\|]*)\|\|\|', e)
            if it_search:
                #beginning position of token offset
                #b_pos = int(it_search.group(1))
                #ending position of token offset
                #e_pos = int(it_search.group(2))
                #annotation type ERRANT produced(M,R,U)
                ann_type = it_search.group(3)
                #word(s) that underwent action
                target = str(it_search.group(4))
                if ann_type == 'M:PRON':
                    #550 Insertions
                    pro_counter_c_m +=1
                    
                    if target == 'I':
                        i_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            i_insert_nsubj += 1
                            
                    elif target == 'me':
                        me_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            me_insert_nsubj += 1
                        elif dep_list[2] == 'dobj':
                            me_insert_dobj +=1
                            
                    elif target == 'you':
                        you_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            you_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            you_insert_dobj +=1
                            
                    elif target == 'it':
                        it_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            it_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            it_insert_dobj +=1
                                
                    elif target == 'It':
                        it_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            it_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            it_insert_dobj +=1
                            
                    elif target == 'IT':
                        it_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            it_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            it_insert_dobj +=1
                            
                    elif target == 'he':
                        he_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            he_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            he_insert_dobj +=1
                            
                    elif target == 'she':
                        she_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            she_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            she_insert_dobj +=1
                            
                    elif target == 'him':
                        him_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            him_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            him_insert_dobj +=1
                            
                    elif target == 'her':
                        her_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            her_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            her_insert_dobj +=1
                            
                    elif target == 'we':
                        we_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            we_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            we_insert_dobj +=1
                        
                    elif target == 'us':
                        us_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            us_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            us_insert_dobj +=1
                    
                    elif target == 'they':
                        they_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            they_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            they_insert_dobj +=1
                            
                    elif target == 'them':
                        them_insert += 1
                        for c in c_d:
                            pro_search = re.search(target + r'\_(PRON)\_' r'(.*)', c)
                            if pro_search:
                                #print(pro_search.group())
                                dep_list = c.split('_')
                                #print(dep_list)
                        if dep_list[2] == 'nsubj':
                            them_insert_nsubj +=1
                        elif dep_list[2] == 'dobj':
                            them_insert_dobj +=1
            
            
    for l in l_d:
        #catpuring all pronouns in learner sentences
        l_search = re.findall(r'\_(PRON)\_', l)
        if l_search:
            l_count = len(l_search)
            pro_counter_l += l_count
        #capturing all pronoun usage 'I' in learner sentences
        l_i_search = re.findall(r'^(?i)(I)\_', l)
        if len(l_i_search) != 0:
            l_i_count = len(l_i_search)
            pro_counter_l_i += l_i_count
        #capturing all pronoun usage 'me' in learner sentences
        l_me_search = re.findall(r'^(?i)(Me)\_', l)
        if len(l_me_search) != 0:
            l_me_count = len(l_me_search)
            pro_counter_l_me += l_me_count
        #capturing all pronoun usage 'you' in learner sentences
        l_you_search = re.findall(r'^(?i)(You)\_', l)
        if len(l_you_search) != 0:
            l_you_count = len(l_you_search)
            pro_counter_l_you += l_you_count
        #capturing all pronoun usage 'It' in learner sentences
        l_it_search = re.findall(r'^(?i)(It)\_(PRON)', l)
        if len(l_it_search) != 0:
            l_it_count = len(l_it_search)
            pro_counter_l_it += l_it_count
        #capturing all pronoun usage 'It' in learner sentences in 'nsubj' position
        l_it_nsubj_search = re.findall(r'^(?i)(It)\_(PRON)\_(nsubj)', l)
        if l_it_nsubj_search:
            l_it_nsubj_search_count = len(l_it_nsubj_search)
            pro_counter_l_it_nsubj += l_it_nsubj_search_count
        #capturing all pronoun usage 'It' in learner sentences in 'nsubjpass' position
        l_it_nsubjpass_search = re.findall(r'^(?i)(It)\_(PRON)\_(nsubjpass)', l)
        if l_it_nsubjpass_search:
            l_it_nsubjpass_search_count = len(l_it_nsubjpass_search)
            pro_counter_l_it_nsubjpass += l_it_nsubjpass_search_count
        #capturing all pronoun usage 'It' in learner sentences in 'dobj' position
        l_it_dobj_search = re.findall(r'^(?i)(It)\_(PRON)\_(dobj)', l)
        if l_it_dobj_search:
            l_it_dobj_search_count = len(l_it_dobj_search)
            pro_counter_l_it_dobj += l_it_dobj_search_count
        #capturing all pronoun usage 'he' in learner sentences
        l_he_search = re.findall(r'^(?i)(He)\_(PRON)\_(.*)', l)
        if l_he_search:
            l_he_search_count = len(l_he_search)
            pro_counter_l_he += l_he_search_count
        #capturing all pronoun usage 'She' in learner sentences
        l_she_search = re.findall(r'^(?i)(She)\_(PRON)\_(.*)', l)
        if l_she_search:
            l_she_search_count = len(l_she_search)
            pro_counter_l_she += l_she_search_count
        #capturing all pronoun usage 'him' in learner sentences
        l_him_search = re.findall(r'^(?i)(Him)\_(PRON)\_(.*)', l)
        if l_him_search:
            l_him_search_count = len(l_him_search)
            pro_counter_l_him += l_him_search_count
        #capturing all pronoun usage 'her' in learner sentences
        l_her_search = re.findall(r'^(?i)(Her)\_(PRON)\_(.*)', l)
        if l_her_search:
            l_her_search_count = len(l_her_search)
            pro_counter_l_her += l_her_search_count
        #capturing all pronoun usage 'we' in learner sentences
        l_we_search = re.findall(r'^(?i)(We)\_(PRON)\_(.*)', l)
        if l_we_search:
            l_we_search_count = len(l_we_search)
            pro_counter_l_we += l_we_search_count
        #capturing all pronoun usage 'us' in learner sentences
        l_us_search = re.findall(r'^(?i)(Us)\_(PRON)\_(.*)', l)
        if l_us_search:
            l_us_search_count = len(l_us_search)
            pro_counter_l_us += l_us_search_count
        #capturing all pronoun usage 'they' in learner sentences
        l_they_search = re.findall(r'^(?i)(They)\_(PRON)\_(.*)', l)
        if l_they_search:
            l_they_search_count = len(l_they_search)
            pro_counter_l_they += l_they_search_count
        #capturing all pronoun usage 'them' in learner sentences
        l_them_search = re.findall(r'^(?i)(Them)\_(PRON)\_(.*)', l)
        if l_them_search:
            l_them_search_count = len(l_them_search)
            pro_counter_l_them += l_them_search_count
            
            
    for c in c_d:
        #capturing all pronouns in corrector sentences
        c_search = re.findall(r'\_(PRON)\_', c)
        if c_search:
            c_count = len(c_search)
            pro_counter_c += c_count
        #capturing pronoun usage 'I' in corrector sentences
        c_i_search = re.findall(r'^(?i)(I)\_', c)
        if len(c_i_search) != 0:
            c_i_count = len(c_i_search)
            pro_counter_c_i += c_i_count
        #capturing all pronoun usage 'me' in corrector sentences
        c_me_search = re.findall(r'^(?i)(Me)\_', c)
        if len(c_me_search) != 0:
            c_me_count = len(c_me_search)
            pro_counter_c_me += c_me_count
        #capturing all pronoun usage 'you' in learner sentences
        c_you_search = re.findall(r'^(?i)(You)\_', c)
        if len(c_you_search) != 0:
            c_you_count = len(c_you_search)
            pro_counter_c_you += c_you_count
        #capturing pronoun usage 'It' in corrector sentences
        c_it_search = re.findall(r'^(?i)(It)\_', c)
        if len(c_it_search) != 0:
            c_it_count = len(c_it_search)
            pro_counter_c_it += c_it_count
         #capturing all pronoun usage 'It' in corrector sentences in 'nsubj' position
        c_it_nsubj_search = re.findall(r'^(?i)(It)\_(PRON)\_(nsubj)', c)
        if c_it_nsubj_search:
            c_it_nsubj_search_count = len(c_it_nsubj_search)
            pro_counter_c_it_nsubj += c_it_nsubj_search_count
        #capturing all pronoun usage 'It' in corrector sentences in 'nsubjpass' position
        c_it_nsubjpass_search = re.findall(r'^(?i)(It)\_(PRON)\_(nsubjpass)', c)
        if c_it_nsubjpass_search:
            c_it_nsubjpass_search_count = len(c_it_nsubjpass_search)
            pro_counter_c_it_nsubjpass += c_it_nsubjpass_search_count
        #capturing all pronoun usage 'It' in corrector sentences in 'dobj' position
        c_it_dobj_search = re.findall(r'^(?i)(It)\_(PRON)\_(dobj)', c)
        if c_it_dobj_search:
            c_it_dobj_search_count = len(c_it_dobj_search)
            pro_counter_c_it_dobj += c_it_dobj_search_count
        #capturing all pronoun usage 'he' in corrector sentences
        c_he_search = re.findall(r'^(?i)(He)\_(PRON)\_(.*)', c)
        if c_he_search:
            c_he_search_count = len(c_he_search)
            pro_counter_c_he += c_he_search_count
        #capturing all pronoun usage 'She' in corrector sentences
        c_she_search = re.findall(r'^(?i)(She)\_(PRON)\_(.*)', c)
        if c_she_search:
            c_she_search_count = len(c_she_search)
            pro_counter_c_she += c_she_search_count
        #capturing all pronoun usage 'him' in corrector sentences
        c_him_search = re.findall(r'^(?i)(Him)\_(PRON)\_(.*)', c)
        if c_him_search:
            c_him_search_count = len(c_him_search)
            pro_counter_c_him += c_him_search_count
        #capturing all pronoun usage 'her' in corrector sentences
        c_her_search = re.findall(r'^(?i)(Her)\_(PRON)\_(.*)', c)
        if c_her_search:
            c_her_search_count = len(c_her_search)
            pro_counter_c_her += c_her_search_count
        #capturing all pronoun usage 'we' in corrector sentences
        c_we_search = re.findall(r'^(?i)(We)\_(PRON)\_(.*)', c)
        if c_we_search:
            c_we_search_count = len(c_we_search)
            pro_counter_c_we += c_we_search_count
        #capturing all pronoun usage 'us' in corrector sentences
        c_us_search = re.findall(r'^(?i)(Us)\_(PRON)\_(.*)', c)
        if c_us_search:
            c_us_search_count = len(c_us_search)
            pro_counter_c_us += c_us_search_count
        #capturing all pronoun usage 'they' in learner sentences
        c_they_search = re.findall(r'^(?i)(They)\_(PRON)\_(.*)', c)
        if c_they_search:
            c_they_search_count = len(c_they_search)
            pro_counter_c_they += c_they_search_count
        #capturing all pronoun usage 'them' in learner sentences
        c_them_search = re.findall(r'^(?i)(Them)\_(PRON)\_(.*)', c)
        if c_them_search:
            c_them_search_count = len(c_them_search)
            pro_counter_c_them += c_them_search_count
    

#general stats
#print(m_pron_counter)
print("Instances of Pronoun usage in learner sentences: ", pro_counter_l)
print("Instances of Pronoun usage in corrector sentences: ", pro_counter_c)
print('Number of corrections in total: ', num, num/len(l_c_dict.keys())*100)
print("Instances of Pronouns 'inserted' by corrector: ", pro_counter_c_m)
print("Learner pronoun accuracy over corrections: ", pro_counter_l/pro_counter_l+pro_counter_c)
print("Pronoun  usage 'I' in learner sentences: ", pro_counter_l_i)
print("Pronoun usage 'I' in corrector sentences: ", pro_counter_c_i)
print("Number of times 'I' was inserted by corrector: ", i_insert)
print("Number of times 'I' was inserted at 'nsubj' position: ", i_insert_nsubj)
print("Pronoun usage 'me' in learner sentences: ", pro_counter_l_me)
print("Pronoun usage 'me' in corrector sentences: ", pro_counter_c_me)
print("Number of times 'me' was inserted by corrector: ", me_insert)
print("Number of times 'me' was inserted by corrector (nsubj): ", me_insert_nsubj)
print("Number of times 'me' was inserted by corrector (dobj): ", me_insert_dobj)
print("Pronoun usage 'you' in learner sentences: ", pro_counter_l_you)
print("Pronoun usage 'you' in corrector sentences: ", pro_counter_c_you)
print("Number of times 'you' was inserted by corrector: ", you_insert)
print("Number of times 'you' was inserted by corrector (nsubj): ", you_insert_nsubj)
print("Number of times 'you' was inserted by corrector (dobj): ", you_insert_dobj)
print("Pronoun usage 'It' in learner sentences: ", pro_counter_l_it)
print("Pronoun usage 'It' in learner sentences (nsubj position): ", pro_counter_l_it_nsubj)
print("Pronoun usage 'It' in learner sentences (dobj position): ", pro_counter_l_it_dobj)
print("Pronoun usage 'It' in corrector sentences: ", pro_counter_c_it)
print("Pronoun usage 'It' in corrector sentences (nsubj position): ", pro_counter_c_it_nsubj)
print("Pronoun usage 'It' in corrector sentences (dobj position): ", pro_counter_c_it_dobj)
print("Number of times 'It' was inserted by corrector: ", it_insert)
print("Number of times 'It' was inserted by corrector (nsubj): ", it_insert_nsubj)
print("Number of times 'It' was inserted by corrector (dobj): ", it_insert_dobj)
print("Pronoun usage 'he' in learner setence: ", pro_counter_l_he)
print("Pronoun usage 'he' in corrector sentence: ", pro_counter_c_he)
print("Number of times 'he' was inserted by corrector: ", he_insert)
print("Number of times 'he' was inserted by corrector (nsubj): ", he_insert_nsubj)
print("Number of times 'he' was inserted by corrector (dobj): ", he_insert_dobj)
print("Pronoun usage 'she' in learner sentence: ", pro_counter_l_she)
print("Pronoun usage 'she' in corrector sentence: ", pro_counter_c_she)
print("Number of times 'she' was inserted by corrector : ", she_insert)
print("Number of times 'she' was inserted by corrector (nsubj): ", she_insert_nsubj)
print("Number of times 'she' was inserted by corrector (dobj): ", she_insert_dobj)
print("Pronoun usage 'him' in learner sentence: ", pro_counter_l_him)
print("Pronoun usage 'him' in corrector sentence: ", pro_counter_c_him)
print("Number of times 'him' was inserted by corrector : ", him_insert)
print("Number of times 'him' was inserted by corrector (nsubj) : ", him_insert_nsubj)
print("Number of times 'him' was inserted by corrector (dobj) : ", him_insert_dobj)
print("Pronoun usage 'her' in learner sentence: ", pro_counter_l_her)
print("Pronoun usage 'her' in corrector sentence: ", pro_counter_c_her)
print("Number of times 'her' was inserted by corrector: ", her_insert)
print("Number of times 'her' was inserted by corrector (nsubj) : ", her_insert_nsubj)
print("Number of times 'her' was inserted by corrector (dobj) : ", her_insert_dobj)
print("Pronoun usage 'we' in learner sentence: ", pro_counter_l_we)
print("Pronoun usage 'we' in corrector sentence: ", pro_counter_c_we)
print("Number of times 'we' was inserted by corrector: ", we_insert)
print("Pronoun usage 'us' in learner sentence: ", pro_counter_l_us)
print("Pronoun usage 'us' in corrector sentence: ", pro_counter_c_us)
print("Number of times 'us' was inserted by corrector (nsubj) : ", us_insert_nsubj)
print("Number of times 'us' was inserted by corrector (dobj) : ", us_insert_dobj)
print("Pronoun usage 'they' in learner sentence: ", pro_counter_l_they)
print("Pronoun usage 'they' in corrector sentence: ", pro_counter_c_they)
print("Number of times 'they' was inserted by corrector: ", they_insert)
print("Number of times 'they' was inserted by corrector (nsubj): ", they_insert_nsubj)
print("Number of times 'they' was inserted by corrector (dobj): ", they_insert_dobj)
print("Pronoun usage 'them' in learner sentence: ", pro_counter_l_them)
print("Pronoun usage 'them' in corrector sentence: ", pro_counter_c_them)
print("Number of times 'them' was inserted by corrector: ", them_insert)
print("Number of times 'them' was inserted by corrector (nsubj): ", them_insert_nsubj)
print("Number of times 'them' was inserted by corrector (dobj): ", them_insert_dobj)

In [ ]:
for item in m_l_object:
    #learner sentence
    l_t = nlp(item[0])
    l_t = [i.text for i in l_t]
    #continue if not right part of corpus (== will yield corrected, >0 uncorrected)
    #if annot_sents[l_t] == 0:
            #continue
    #learner sentence dependencies
    l_d = item[1]
    #corrector sentence
    c_t = str(item[2])
    #corrector sentence dependencies
    c_d = item[3]
    
    for e in item[4]:
            run_search = re.search(r'^A (\d+) (\d+)\|\|\|([^\|]+)\|\|\|([^\|]*)\|\|\|', e)
            if run_search:
                #beginning position of token offset
                b_pos = int(run_search.group(1))
                #ending position of token offset
                e_pos = int(run_search.group(2))
                #annotation type ERRANT produced(M,R,U)
                ann_type = run_search.group(3)
                #word(s) that underwent action
                target = str(run_search.group(4))
                if target == '.':
                    print(l_t[b_pos])
                        #if item == ',':
                            #print(item)
                            #print(l_t,c_t,ann_type)